## --- Day 9: Smoke Basin ---
These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

2199943210
3987894921
9856789892
8767896789
9899965678
Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

Your puzzle answer was 480.

## --- Part Two ---
Next, you need to find the largest basins so you know what areas are most important to avoid.

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size 3:

2199943210
3987894921
9856789892
8767896789
9899965678
The top-right basin, size 9:

2199943210
3987894921
9856789892
8767896789
9899965678
The middle basin, size 14:

2199943210
3987894921
9856789892
8767896789
9899965678
The bottom-right basin, size 9:

2199943210
3987894921
9856789892
8767896789
9899965678
Find the three largest basins and multiply their sizes together. In the above example, this is 9 * 14 * 9 = 1134.

What do you get if you multiply together the sizes of the three largest basins?

Your puzzle answer was 1045660.

Both parts of this puzzle are complete! They provide two gold stars: **

In [50]:
import numpy as np
from scipy import signal 
from collections import namedtuple

def read_file(filename):
    # read file
    with open(filename) as f:
        lines = f.readlines()
    output = [str.rstrip(line) for line in lines]    
    return output

In [190]:
if False:
    data = read_file('./day9_test.txt')
else:
    data = read_file('./day9.txt')

big_array = []
for row in data:
    make_array = []
    for val in row:
        make_array.append(float(val))
    big_array.append(np.array(make_array))

data = np.array(big_array)

In [191]:
def peaks(sig):
    """
    detects peaks and valleys as defined by the first derivative changing signs
    0  : no peak
    2  : peak second derivative positve
    -2 : peak second derivative negative
    Parameters
    ----------
    signal : ndarray 
        input signal to be analyzed
    Returns
    -------
    peak : ndarray
        list of indicies corresponding to peaks/valleys found in the time-domain of the signal
    """

    peak = signal.lfilter([-1, 1], [1], np.sign(np.diff(sig)))
    peak[0] = 0
    peak = np.append(peak, [0])
    return peak


In [192]:
border = np.zeros([data.shape[0]+2, data.shape[1]+2]) + 9
border[1:-1,1:-1] = data

rows = []
for val in border:
    rows.append(peaks(val))
    
cols = []
for val in border.T:
    cols.append(peaks(val))
    
out = np.array(rows) + np.array(cols).T    

In [193]:
int(sum(border[out == -4]+1))

480

In [194]:
index = np.where(out == -4)

In [195]:
Point = namedtuple('Point', ['x', 'y'])

In [196]:
basins = []
for val in zip(index[0], index[1]):
    basins.append(Point(val[0],val[1]))

In [197]:

def intersection(a,b):
    return list(set(a).intersection(b))

def not_intersection(a,b):
    return list(set(a).symmetric_difference(b))

def find_candidates(p,d):
    mask = [Point(p.x-1,p.y),Point(p.x+1,p.y),Point(p.x,p.y-1),Point(p.x,p.y+1)]
    return [q for q in mask if d[q.x,q.y] != 9]

def basin(p,d):
    
    found.append(p)
    new = find_candidates(p,d)
    new = not_intersection(new,intersection(new, found))

    candidates.extend(new)
    if not candidates:
        return 1
    else:
        winner = candidates.pop(0)
        return(1 + basin(winner,d))

In [198]:
basins[2]

Point(x=1, y=74)

In [199]:
border

array([[9., 9., 9., ..., 9., 9., 9.],
       [9., 7., 6., ..., 6., 5., 9.],
       [9., 6., 5., ..., 9., 3., 9.],
       ...,
       [9., 6., 7., ..., 8., 7., 9.],
       [9., 7., 9., ..., 9., 8., 9.],
       [9., 9., 9., ..., 9., 9., 9.]])

In [200]:
deep = []
for val in basins:
    found = []
    candidates = []
    basin(val,border)
    deep.append(len(set(found)))

In [201]:
out = 1
for count in range(3):
    max_val = max(deep)
    out = out * max_val
    idx = [ii for ii, val in enumerate(deep) if val == max_val]
    deep.pop(idx[0])
    

In [202]:
out

1045660